In [ ]:
filePath = "..\\..\\Files\\"
filePathPersonalLists = "..\\..\\Files\\Besökslistor\\"

In [ ]:
# Check that each file isn't the same size
import os

def list_files_by_size(folder_path):
    # List to store file names and their sizes
    files_info = []
    
    # Iterate through files in the folder
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        # Check if it's a file
        if os.path.isfile(file_path):
            file_size = os.path.getsize(file_path)
            files_info.append((file_name, file_size))
    
    # Sort files by size (ascending order)
    sorted_files = sorted(files_info, key=lambda x: x[1])
    
    # Print sorted files with sizes
    for file_name, file_size in sorted_files:
        print(f"{file_name}: {file_size} bytes")

list_files_by_size(filePathPersonalLists)

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import re
from collections import Counter
from datetime import datetime
from pprint import pprint
import math

### Scan for pnr
#\b\d{12}\b|\d{8}-\w{4}

In [ ]:
reloadDataFiles = True

# Import files

In [ ]:
def LoadDrFile(initials):
    drDfVisists = pd.read_excel(f"{filePathPersonalLists}{initials}_besök.xlsx")
    drDfPhone = pd.read_excel(f"{filePathPersonalLists}{initials}_telefon.xlsx")
    drDfAdmin = pd.read_excel(f"{filePathPersonalLists}{initials}_indirekt.xlsx")
    drDfDistans = pd.read_excel(f"{filePathPersonalLists}{initials}_distans.xlsx")
    drDfDistans.loc[:, "Variabelnamn i M4"] = "Telefon"
    
    # Append all DataFrames into one
    drDf = drDfVisists.append(drDfPhone, ignore_index=True)
    drDf = drDf.append(drDfDistans, ignore_index=True)
    drDf = drDf.append(drDfAdmin, ignore_index=True)
    
    # Limit columns and filter out rows with null in 'Personnr'
    limitColsDf = drDf.loc[:, ["Personnr", "Variabelnamn i M4", "Besöksdatum"]]
    nonNaNDf = limitColsDf.loc[limitColsDf['Personnr'].notnull()].copy()  # Explicitly create a copy
    nonNaNDf["Personnr"] = nonNaNDf["Personnr"].apply(lambda x: f"{int(float(x)):012}" if isinstance(x, (float, int)) else f"{x:>012}")
    
    # Add the 'Initials' column
    nonNaNDf.loc[:, "Initials"] = initials
    
    return nonNaNDf


In [ ]:
if reloadDataFiles:
    JK = LoadDrFile("JK")
    MT = LoadDrFile("MT")
    LW = LoadDrFile("LW")
    RA = LoadDrFile("RA")
    MS = LoadDrFile("MS")
    MN = LoadDrFile("MN")
    DH = LoadDrFile("DH")
    AE = LoadDrFile("AE")
    IW = LoadDrFile("IW")
    LYY = LoadDrFile("LYY")

In [ ]:
# if reloadDataFiles:
#     print(RA.loc[800, :])

In [ ]:
if reloadDataFiles:
    listingOriginalDf = pd.read_csv(f"{filePath}Listing_2024-12-11.csv", sep=";")
    listingDf = listingOriginalDf.loc[:, ["Pnr", "Vårdgivare"]]
    listingDf.head(2)

In [ ]:
def read_tab_separated_file(file_path):
    result_dict = {}
    
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            for line in file:
                # Ignorera tomma rader
                if line.strip():
                    key, value = line.strip().split('\t')
                    result_dict[key] = value
    except FileNotFoundError:
        print(f"Filen {file_path} kunde inte hittas.")
    except ValueError:
        print("Fel format i filen. Kontrollera att varje rad innehåller exakt en tab.")

    return result_dict


In [ ]:
DrToInitials = read_tab_separated_file(filePath+"Permanent\\DrToInitials_ListingAnalysis.txt")


In [ ]:
if reloadDataFiles:
    listingDf.rename(columns={"Pnr": "Personnr", "Vårdgivare": "Listing"}, inplace=True)
    listingDf["Listing"] = listingDf["Listing"].apply(lambda name: DrToInitials[name] )
    listingDf.head()

In [ ]:
listing_counter = Counter(listingDf["Listing"])
for initials in [ "MT", "JK", "RA", "LW", "MS", "H", "AE", "MN", "DH", "HC"]:
    print(f"{initials}\t{listing_counter[initials]}")

In [ ]:
if reloadDataFiles:
    listingDict = dict(zip(listingDf["Personnr"], listingDf["Listing"]))

## Lägg ihop alla besök med listad läkar som egen kolumn

In [ ]:
if reloadDataFiles:
    allContacts = JK.append(MT, ignore_index=True)
    allContacts = allContacts.append(LW, ignore_index=True)
    allContacts = allContacts.append(RA, ignore_index=True)
    allContacts = allContacts.append(MS, ignore_index=True)
    allContacts = allContacts.append(MN, ignore_index=True)
    allContacts = allContacts.append(DH, ignore_index=True)
    allContacts = allContacts.append(AE, ignore_index=True)
    allContacts = allContacts.append(LYY, ignore_index=True)
    allContacts = allContacts.append(IW, ignore_index=True)

    allContacts = allContacts.copy()

In [ ]:
if reloadDataFiles:
    allContacts["Personnr"] = allContacts["Personnr"].str.replace(r"(\d{8})(\w{4})", r"\1-\2")

In [ ]:
if reloadDataFiles:
    print(set(allContacts["Variabelnamn i M4"]))

In [ ]:
visitTypeDict = {
    'Indirekt kontakt': "adm",
    'Planerade mottagningsbesök': "visit",
    'Planerat hembesök': "visit",
    'Telefon': "tel"
}

In [ ]:
if reloadDataFiles:
    allContacts.rename(columns = {"Variabelnamn i M4" : "VisitType"}, inplace=True)
    allContacts["VisitType"] = allContacts["VisitType"].apply(lambda x: visitTypeDict[x])
#     print(allContacts.head(1))

In [ ]:
def checkListing(personnr):
    if not personnr in listingDict:
        return "unknown"
    
    return listingDict[personnr]

In [ ]:
if reloadDataFiles:
    allContacts["Listing"] = allContacts["Personnr"].apply( checkListing )

In [ ]:
if reloadDataFiles:
    allContacts.to_excel(f"{filePath}allContacts.xlsx", index=False)
else:
    allContacts = pd.read_excel(f"{filePath}allContacts.xlsx")
    try:
        allContacts.drop(columns=["Unnamed: 0", "Unnamed: 0.1"], inplace=True)
    except:
        print("Unnamed columns not found")

In [ ]:
if reloadDataFiles:
    listingDf["Visits (n)"] = listingDf["Personnr"].apply(
        lambda personnr: len(allContacts.loc[ 
            (allContacts["Personnr"] == personnr) & 
            (allContacts["VisitType"] == "visit")].index))

    listingDf["Tel (n)"] = listingDf["Personnr"].apply(
        lambda personnr: len(allContacts.loc[ 
            (allContacts["Personnr"] == personnr) & 
            (allContacts["VisitType"] == "tel")].index))

    listingDf["Adm (n)"] = listingDf["Personnr"].apply(
        lambda personnr: len(allContacts.loc[ 
            (allContacts["Personnr"] == personnr) & 
            (allContacts["VisitType"] == "adm")].index))
    
    listingDf.to_excel(f"{filePath}listingDf.xlsx")
else:
    listingDf = pd.read_excel(f"{filePath}listingDf.xlsx")
    listingDf.drop(columns=["Unnamed: 0"], inplace=True)

### Lägg till antalet besök per listad pat i listingDf

In [ ]:
listingDf.loc[:, ["Listing", "Visits (n)", "Tel (n)", "Adm (n)"]].head(2)

In [ ]:
unlistedDf = listingDf.loc[ listingDf["Listing"] == "HC" ]
print(len(unlistedDf.index))


In [ ]:
#allContacts.loc[ allContacts["Personnr"] == "some_nr" ]

## Ta fram aktiva patienter (listad på HC som ej haft besök sista åren)

In [ ]:
hasVisitSet = set( allContacts.loc[ allContacts["VisitType"] == "visit" ].loc[ : , "Personnr"] )

In [ ]:
print(f"There were {len(allContacts.index)} contacts in total.")
print(f"There were {len(allContacts.loc[ (allContacts['Listing'] == 'HC') ].index)} contacts from unlisted patients.")
unlistedHasContactSet = set( allContacts.loc[ (allContacts["Listing"] == "HC") ].loc[ : , "Personnr"] )
print(f"{len(unlistedHasContactSet)} unlisted patients had som form of contact.")
unlistedHasVisitSet = set( allContacts.loc[ (allContacts["Listing"] == "HC") & (allContacts["VisitType"] == "visit") ].loc[ : , "Personnr"] )
print(f"{len(unlistedHasVisitSet)} unlisted patients had a physical visit.")
unlistedHasTelSet = set( allContacts.loc[ (allContacts["Listing"] == "HC") & (allContacts["VisitType"] == "tel") ].loc[ : , "Personnr"] )
print(f"{len(unlistedHasTelSet)} unlisted patients had a tel appointment.")

In [ ]:
print(f"There are {len(listingDf.loc[ listingDf['Listing'] == 'HC' ].index)} unlisted patients in total")

In [ ]:
unlistedNoContactSet = listingDf.loc[ (listingDf["Listing"] == "HC") & ~(listingDf["Personnr"].isin(unlistedHasContactSet)) ]
#print(unlistedNoContactSet[:5])
print(f"{len(unlistedNoContactSet)} unlisted patietns had no contacts")

# Beräkningar

#### Använd:
* unlistedContacts - DF med alla olistade patienter som haft besök
* unlistedNoContactDf - DF med patienter som är olistade och inte har haft några kontakter
* unlistedContactsDf - DF med alla kontakter med olistade patienter
* unlistedHasContactSet - set med alla olistade patienter som har en kontakt

### Patient to visits per doctor df

In [ ]:
initialsList = ["MT", "JK", "LW", "RA", "MS", "MN", "DH", "AE", "LYY", "IW"]
visitTypes = ["visit", "tel", "adm"]

In [ ]:
countAndScoreColNames = []
for initials in initialsList:
    for visitType in visitTypes:
        countAndScoreColNames.append(f"{initials}_{visitType}")
        countAndScoreColNames.append(f"{initials}_{visitType}_score")
    countAndScoreColNames.append(f"{initials}_score")
countAndScoreColNames.append(f"total_score")
countAndScoreColNames

In [ ]:
def countPatientsContactsWithDr(personnr, initials, visitType):
    return len(allContacts.loc[ (allContacts["Personnr"] == personnr) & 
                                   (allContacts["Initials"] == initials) &
                                   (allContacts["VisitType"] == visitType)].index)

#### Vikta besöken efter hur längesen de skedde.

In [ ]:
date = datetime.strptime("2023-11-01", '%Y-%m-%d')
originDate = datetime.strptime("2024-12-08", '%Y-%m-%d')
dateDiff = (originDate - date).days
2 - (dateDiff/450)

In [ ]:
# Weight factor where a visit which happened just now is worth 2 points
# one which happened 1 year ago 1.16 points
# and a bit less than 2 years ago 0.5 point.

def weightFactorFromDate(dateString):
    date = datetime.strptime(dateString, '%Y-%m-%d')
    originDate = datetime.strptime("2024-12-08", '%Y-%m-%d')
    dateDiff = (originDate - date).days
    return 2 - (dateDiff/450)
    
print(weightFactorFromDate("2024-11-15"))
print(weightFactorFromDate("2024-05-15"))
print(weightFactorFromDate("2024-01-01"))
print(weightFactorFromDate("2023-11-15"))
print(weightFactorFromDate("2023-05-15"))
print(weightFactorFromDate("2023-01-01"))

visitTypeWeightDict = {
    "visit": 1,
    "tel": 0.35,
    "adm": 0.2,
}

In [ ]:
def calculateRowScore(row):
    dateWeightFactor = weightFactorFromDate(row["Besöksdatum"])
    return visitTypeWeightDict[ row["VisitType"] ] * dateWeightFactor

testRow = allContacts.loc[ list(allContacts.index)[100] , :]
pprint(testRow.loc[list(set(testRow.index) - set(["Personnr"]))])
print(f"Score: {calculateRowScore( testRow )}")

In [ ]:
allContacts["rowScore"] = allContacts.apply(calculateRowScore, axis=1)

In [ ]:
allContacts.loc[:, ["VisitType", "Besöksdatum", "Initials", "Listing", "rowScore"]].head(2)

In [ ]:
allContacts.columns

In [ ]:
def createPatientRowOfVisitsPerDr(personnr):
    patientsIndividualDf = allContacts.loc[ allContacts["Personnr"] == personnr ]
    
    row = {"Personnr": personnr}
    totalScore = 0
    
    for initials in initialsList:
        
        drIndividualDf = patientsIndividualDf.loc[ patientsIndividualDf["Initials"] == initials ]
        
        individualScore = 0
        
        for visitType in visitTypes:
            
            visitTypeDf = drIndividualDf.loc[ drIndividualDf["VisitType"] == visitType ]
            row[f"{initials}_{visitType}"] = len( visitTypeDf.index )
            score = visitTypeDf["rowScore"].sum()
            row[f"{initials}_{visitType}_score"] = score
            individualScore += score
            
        row[f"{initials}_score"] = individualScore
        totalScore += individualScore
    row[f"total_score"] = totalScore
    return row

In [ ]:
if reloadDataFiles:
    patientVisitPerDrList = []
    limit_proportion = -1
    i = 0
    for personnr in set(listingDf["Personnr"]):
        i += 1
        if limit_proportion > 0 and i > limit_proportion:
            break
        
        patientVisitPerDrList.append(createPatientRowOfVisitsPerDr(personnr))
        
        
    listedPatientScoreDf = pd.DataFrame(patientVisitPerDrList, columns=["Personnr"] + countAndScoreColNames)
    listedPatientScoreDf.to_excel(filePath+"listedPatientScoreDf.xlsx", index=False)
else:
    listedPatientScoreDf = pd.read_excel(filePath+"listedPatientScoreDf.xlsx", index=False)

listedPatientScoreDf = listedPatientScoreDf
listedPatientScoreDf.loc[:, countAndScoreColNames].head()

In [ ]:
def countDrAmountOfPatientsForEachVisitAmount(initials):
    counter = Counter(listedPatientScoreDf[f"{initials}_visit"])
    return dict(counter)
    #return sorted(counter.items())

In [ ]:
visistsPerDrDict = {}
for initials in initialsList:
    visistsPerDrDict[initials] = countDrAmountOfPatientsForEachVisitAmount(initials)
    
visistsPerDrDict

In [ ]:
visistsPerDrList = []
for i in range(6):
    #row = {"Visits (n)": i}
    row = {}
    for initials in initialsList:
        if i in visistsPerDrDict[initials]:
            row[initials] = visistsPerDrDict[initials][i]
        else:
            row[initials] = 0
    visistsPerDrList.append(row)
    
unlistedVisistsPerDrDf = pd.DataFrame(visistsPerDrList, columns=initialsList)
unlistedVisistsPerDrDf

### Extremt aktiva till JK och MT

In [ ]:
# Creating the histogram
plt.figure(figsize=(10, 6))
listedPatientScoreDf["total_score"].hist(bins=range(0, 17), edgecolor="black")  # Bins from 0 to 16
plt.title("Histogram of Total Score")
plt.xlabel("Total Score")
plt.ylabel("Frequency")
plt.xticks(range(0, 17))  # Set x-axis ticks to match the bin edges
plt.xlim(0, 16)  # Limit x-axis from 0 to 16
plt.grid(axis='y', linestyle='--', alpha=0.7)  # Optional: Add gridlines for better readability
plt.show()

### Most active patients

In [ ]:
def ageFromPersonnr(personnrStr):
    match = re.search(r"(\d{4})", personnrStr)
    year = int(match[1])
    return 2024 - year
    
ageFromPersonnr("19870323-abcd")

In [ ]:
ageAndScoreDf = listedPatientScoreDf.loc[:, ["Personnr", "total_score"]].copy()

In [ ]:
ageAndScoreDf["age"] = ageAndScoreDf["Personnr"].apply(ageFromPersonnr)

## Age distribution of patients

In [ ]:
ageAndScoreDf["age"].hist()

### Contact score per age

In [ ]:
test = np.array([1,0,0,0.25])
test.reshape(1, -1)
ageAndScoreDf.plot.scatter(x="age", y="total_score", c=test )

In [ ]:
scoreLimitForHeavy = 5
heavySet = set( listedPatientScoreDf.loc[ listedPatientScoreDf["total_score"] >= scoreLimitForHeavy ].loc[ : ,"Personnr"] )
heavyN = len(heavySet)
listedN = len(listingDf.index)
print(f"Heavy ({heavyN}) / listed {listedN} = {100 * heavyN / listedN} %")

#### Extremely active from a particular dr

In [ ]:
def GetExtremelyActiveForDr(initials, scoreLimit, otherAllowedFraction):
    resultDf = listedPatientScoreDf.loc[ (listedPatientScoreDf[f"{initials}_score"] > scoreLimit) |
                                           (listedPatientScoreDf[f"{initials}_visit"] >= 3) ]

    otherInitialsList = [x for x in initialsList if not x == initials]

    for otherInitials in otherInitialsList:
        resultDf = resultDf.loc[ resultDf[f"{otherInitials}_score"] < resultDf[f"{initials}_score"] * otherAllowedFraction ]

    return resultDf.loc[:, [
                 'Personnr',
                 f'{initials}_visit',
                 f'{initials}_visit_score',
                 f'{initials}_tel',
                 f'{initials}_tel_score',
                 f'{initials}_adm',
                 f'{initials}_adm_score',
                 f'{initials}_score',
                 'total_score',]]

In [ ]:
def CheckIfOthersScoreExceeds(original_initials, score_row, limit_proportion=1):
    other_initials = initialsList.copy()
    other_initials.remove(original_initials)
    score = score_row[f"{original_initials}_score"]
    score_limit = score * limit_proportion
    exceeding = []

    for initials in initialsList:
        if score_row[f"{initials}_score"] > score_limit:
            exceeding.append(initials)
    
    if len(exceeding) > 0:
        return exceeding
    else:
        return None
    
test_df = listedPatientScoreDf.loc[1:3, [f"{x}_score" for x in initialsList]]
test_df.apply(lambda x: CheckIfOthersScoreExceeds("LW", x), axis="columns")

In [ ]:
extremelyActiveJK = GetExtremelyActiveForDr("JK", 5, 0.4)
len(extremelyActiveJK.index)

In [ ]:
extremelyActiveMT = GetExtremelyActiveForDr("MT", 5, 0.4)
len(extremelyActiveMT.index)

### Patients with contacts but no visits?

In [ ]:
contactNoVisitDf = unlistedDf.loc[ (unlistedDf["Visits (n)"] == 0) & ((unlistedDf["Tel (n)"] > 0) | (unlistedDf["Adm (n)"] > 0)) ]
print(len(contactNoVisitDf))
contactNoVisitDf.loc[:, ["Listing", "Visits (n)", "Tel (n)", "Adm (n)"]].head()

# Date functions

In [ ]:
def string_to_date(date):
    """
    Converts a date string in the format 'YYYY-MM-DD' to a datetime object.

    Args:
        date (str): Date string in the format 'YYYY-MM-DD'.

    Returns:
        datetime: A datetime object representing the date.
    """
    if isinstance(date, str):
        return datetime.strptime(date, "%Y-%m-%d")
    return date

def date_to_string(date):
    """
    Converts a datetime object to a string in the format 'YYYY-MM-DD'.

    Args:
        date (datetime): A datetime object.

    Returns:
        str: A string representation of the date in the format 'YYYY-MM-DD'.
    """

    if isinstance(date, str):     
        return date.strftime("%Y-%m-%d")
    return date

def date_in_range(date, date_start, date_end):
    """
    Checks if a date is within a specified range.

    Args:
        date (str or datetime): The date to check.
        date_start (str or datetime): The start of the range.
        date_end (str or datetime): The end of the range.

    Returns:
        bool: True if the date is within the range, False otherwise.
    """
    # Convert inputs to datetime if they are strings
    if isinstance(date, str):
        date = string_to_date(date)
    if isinstance(date_start, str):
        date_start = string_to_date(date_start)
    if isinstance(date_end, str):
        date_end = string_to_date(date_end)
    
    return date_start <= date <= date_end

In [ ]:
class DateInRangeChecker:
    def __init__(self, start_date, end_date):
        self.start_date = string_to_date(start_date)
        self.end_date = string_to_date(end_date)

    def check(self, date):
        return date_in_range(date, self.start_date, self.end_date)

test_checker = DateInRangeChecker("2024-01-01", "2024-02-01")
print(test_checker.check("1999-01-01"))
print(test_checker.check("2024-01-03"))


# Load waitlist patients

In [ ]:
def format_waitlist_personnr(personnr):
    """
    Formats the person number by determining the correct century.

    Args:
        personnr (str): The person number in the format "YYMMDD-XXXX" or "YYMMDD+XXXX".
        originDate (datetime): The reference date to determine the century.

    Returns:
        str: The formatted person number in the format "YYYYMMDD-XXXX".
    """
    # Extract the year, month, and day from the personnr
    date_part, suffix = personnr.split("-") if "-" in personnr else personnr.split("+")
    year = int(date_part[:2])
    month_day = date_part[2:]

    # Determine the full year
    current_century = originDate.year // 100
    full_year = current_century * 100 + year

    # If there's a '+' or if the year is over a century in the future, adjust the century
    if "+" in personnr or full_year > originDate.year:
        full_year -= 100

    # Format the full date and return
    formatted_personnr = f"{full_year:04d}{month_day}-{suffix}"
    return formatted_personnr

# Example usage
origin_date = datetime(2023, 11, 16)
personnr = "890101-1234"
formatted = format_waitlist_personnr(personnr)
print(formatted)  # Outputs: "19890101-1234"

personnr = "210101+1234"
formatted = format_waitlist_personnr(personnr)
print(formatted)  # Outputs: "19210101-1234"

personnr = "210101-1234"
formatted = format_waitlist_personnr(personnr)
print(formatted)  # Outputs: "20210101-1234"

In [ ]:
DrToInitialsForWatilist = read_tab_separated_file(filePath+"Permanent\\DrToInitialsForWaitlist_ListingAnalysis.txt")

In [ ]:
wait_list_df = pd.read_excel(filePath+"waiting_list.xlsx")
wait_list_df["Personnr"] = wait_list_df["Personnr"].apply(format_waitlist_personnr)
filtered_wait_list_df = wait_list_df.loc[ wait_list_df["staff_member"].isin(DrToInitialsForWatilist) ].copy()
filtered_wait_list_df["staff_member"] = filtered_wait_list_df["staff_member"].apply(lambda x : DrToInitialsForWatilist[x])
filtered_wait_list_df.loc[:,["age", "book_from", "book_not_later_than", "staff_member", "visit_type", "waited_since"]].head()
wait_list_df = filtered_wait_list_df
len(wait_list_df)

For a given doctor and a given period of time
1. For each upcoming month keep only an amount corresponding to N patiens per week.
2. Choose to keep patients which had a visit och telephone appointment within 6 months.
3. Prioritize keeping high score patients.
4. Set up the rest on a list for relisting

In [ ]:
def get_wait_list_patients(df, initials, start_date, end_date):
    checker = DateInRangeChecker(start_date, end_date)
    return df.loc[ (df["book_not_later_than"].apply(checker.check)) & (df["staff_member"] == initials) ]

# Functions for dividing the patients

In [ ]:
differences = {
    "MT": -57,
    "JK": -224,
    "LW": 52,
    "RA": 20,
    "MS": 49,
    "MN": -106,
    "DH": -44,
    "AE": 108,
    "ST": 0,
    "H": 203,
}

proportions_of_new = {
    "MT": 0,
    "JK": 0,
    "LW": 0.1198891591,
    "RA": 0.04541084658,
    "MS": 0.1129333258,
    "MN": 0,
    "DH": 0,
    "AE": 0.2505231013,
    "ST": 0,
    "H": 0.4712435673,
}

time_periods = [
    { "start": "2015-01-01", "end": "2024-12-31" },
    { "start": "2025-01-01", "end": "2025-03-31" },
    { "start": "2025-04-01", "end": "2025-08-31" },
    { "start": "2025-09-01", "end": "2050-12-31" },
]

### Function to do the dividing by:
1. Stratfy by quarter.
2. Exclude the 20 % patients with the highest scores for current owner (as long as another docter doesn't exceed it)
3. Fill LW, RA, MS and AE with the ones they had the highest scores for them (as long as another doctor which is still left, doesn't exceed it).
4. Add the remaining to H with a random sample.

In [ ]:
listedPatientScoreDf.columns

In [ ]:
sorted_cached_score_dfs_per_dr = {}

for initials in initialsList:
    sorted_cached_score_dfs_per_dr[initials] = listedPatientScoreDf.sort_values(by=f"{initials}_score", axis="index", ascending=False)

# H should just have a random list:
sorted_cached_score_dfs_per_dr["H"] = listedPatientScoreDf.sample(frac=1).reset_index(drop=True)

sorted_cached_score_dfs_per_dr["H"].loc[:, ["JK_score", "total_score"]].head()

In [ ]:
def listsHaveAnyMatches(list1, list2):
    return bool(set(list1) & set(list2))

In [ ]:
wait_list_df.columns

In [ ]:
to_from_row_copy_columns = [f"{x}_score" for x in ["JK", "MT", "RA", "AE", "LW", "MS"]] + ["total_score"]
to_from_row_columns = ["Personnr", "From", "To", "Category"] + to_from_row_copy_columns

def create_to_from_row(row, original_initials, new_initials, category="waiting list"):
    new_row = {
        "Personnr" : row["Personnr"],
        "From": original_initials,
        "To": new_initials,
        "Category": category,
    }
    for col in to_from_row_copy_columns:
        new_row[col] = row[col]
    return new_row

def create_to_from_row_empty(personnr, original_initials, new_initials, category="waiting list"):
    new_row = {
        "Personnr" : personnr,
        "From": original_initials,
        "To": new_initials,
        "Category": category,
    }
    for col in to_from_row_copy_columns:
        new_row[col] = 0
    return new_row

# Waitinglist patients

In [ ]:
def divide(original_initials, time_periods):
    time_periods = time_periods.copy()

    print(f"=========== {original_initials}===========")
    wl = wait_list_df.loc[ wait_list_df["staff_member"] == original_initials ].copy()
    total_before_dupl_remove = len(wl.index)
    print("Original patients", total_before_dupl_remove)
    wl.drop_duplicates("Personnr", inplace=True)
    total_patients = len(wl.index)
    print("After dupl remove", total_patients)
    total_n_to_move = abs(differences[original_initials]) - (total_before_dupl_remove - total_patients)
    if original_initials in ["MN", "DH"]:
        total_n_to_move = total_patients
    original_proportion_to_keep = (total_patients - total_n_to_move) / total_patients
    print(f"Original doctor will keep {original_proportion_to_keep:0.1%}")
    print("Total n to move: ", total_n_to_move)
    period_wls = []
    period_proportions = []

    original_sorted_score_df = sorted_cached_score_dfs_per_dr[original_initials]

    # Split into time periods
    for i in range(4):
        period_wl = get_wait_list_patients(wl, original_initials, time_periods[i]["start"], time_periods[i]["end"])
        period_wls.append(period_wl)
        period_proportions.append(len(period_wl.index) / total_patients)

    moves = []

    # Divide patients for each time period
    for i in range(4):
        period_wl = period_wls[i]
        print(f"""\n===Time period 1 ({time_periods[i]["start"]} - {time_periods[i]["end"]})===""")
        # Calculate how many to divide:
        total_period = len(period_wl.index)
        period_proportion = period_proportions[i]
        period_move_n = round(total_n_to_move * period_proportion)
        print(f"Move {period_move_n} out of {len(period_wl.index)} this period")
        personnr_set = set(period_wl["Personnr"])

        # First exclude the patients with the highest score for the original dr.
        # original_reserved_n = round(min(total_period - period_move_n, total_period * 0.2))
        original_reserved_n = math.floor(original_proportion_to_keep * total_period)
        print(f"Will try to reserve {original_reserved_n} for {original_initials}")

        original_dr_sorted_filtered_scores = original_sorted_score_df.loc[ original_sorted_score_df["Personnr"].isin(personnr_set) ]
        reserved_count = 0
        reserved_set = set()

        for index, row in original_dr_sorted_filtered_scores.iterrows():
            if CheckIfOthersScoreExceeds(original_initials, row, 1.01):
                continue
            
            moves.append(create_to_from_row(row, original_initials, original_initials))
            reserved_count += 1
            personnr = row["Personnr"]
            reserved_set.add(personnr)
            personnr_set.remove(personnr)

            if reserved_count >= original_reserved_n:
                break

        print(f"""Reserved: {reserved_count}: """, reserved_set)
        print(f"""{len(personnr_set)} remains""")

        # Then transfer the top patients for each of LW, RA, MS and AE
        # 1. Make sure they don't get too many for this period
        patients_n_per_dr = {}
        period_try_move_n = 0
        patients_did_move_n_per_dr = {}

        all_receiving_initials = ["RA", "AE", "LW", "MS", "H"]
        initials_left_to_receive = ["RA", "AE", "LW", "MS", "H"]

        for initials in all_receiving_initials:
            move_n = round(period_move_n * proportions_of_new[initials])
            moved_so_far = 0
            period_try_move_n += move_n
            patients_n_per_dr[initials] = move_n

            sorted_score_list_for_dr = sorted_cached_score_dfs_per_dr[initials]
            filtered_on_relevant_patients = sorted_score_list_for_dr.loc[ sorted_score_list_for_dr["Personnr"].isin(personnr_set) ]
            print(f"Sanity check - there are {len(filtered_on_relevant_patients.index)} patients to choose from for {initials}")
            for index, row in filtered_on_relevant_patients.iterrows():
                exeeding = None
                if initials != "H":
                    exeeding = CheckIfOthersScoreExceeds(initials, row, 1.01)

                # Only add the patient if it's score is not exceeded by anyone else who will receive patients.
                if not exeeding or not listsHaveAnyMatches(exeeding, initials_left_to_receive):
                    personnr = row["Personnr"]
                    moves.append(create_to_from_row(row, original_initials, initials))
                    personnr_set.remove(personnr)
                    moved_so_far += 1

                    if moved_so_far == move_n:
                        break

            # If not enough were found, iterate a second time and add any, regardless if someone else exceeds.
            filtered_on_relevant_patients = filtered_on_relevant_patients.loc[ filtered_on_relevant_patients["Personnr"].isin(personnr_set) ]
            if moved_so_far < move_n:
                for index, row in filtered_on_relevant_patients.iterrows():
                    personnr = row["Personnr"]
                    moves.append(create_to_from_row(row, original_initials, initials))
                    personnr_set.remove(personnr)
                    moved_so_far += 1

                    if moved_so_far == move_n:
                        break

            patients_did_move_n_per_dr[initials] = moved_so_far

            initials_left_to_receive.remove(initials)

        print(f"After dividing between new doctors, {len(personnr)} patients remains which were evidently not in the filtered_on_relevant_patients df")
        extra_move_to_h = 0

        remaining_to_last_receiver = patients_n_per_dr["H"] - patients_did_move_n_per_dr["H"]
        # Any remaining patients are given to H or kept by original:
        # (there may be patients left to give to H because they are not in the score df)
        original_dr_sorted_filtered_scores = original_dr_sorted_filtered_scores.loc[ original_dr_sorted_filtered_scores["Personnr"].isin(personnr_set) ]
        for personnr in personnr_set:
            target = "H" if remaining_to_last_receiver > 0 else original_initials
            # Find its row:
            single_person_df = original_dr_sorted_filtered_scores.loc[ original_dr_sorted_filtered_scores["Personnr"] == personnr]

            if not single_person_df.empty:
                row = single_person_df.iloc[0]
                moves.append(create_to_from_row(row, original_initials, target))
            else:
                moves.append(create_to_from_row_empty(personnr, original_initials, target))

            if target == "H":
                remaining_to_last_receiver -= 1
                extra_move_to_h += 1




        print(f"Tried to move a total of {period_try_move_n}:")
        succeeded_count_str = "SUCCEEDED: "
        for initials in all_receiving_initials:
            succeeded_count_str += f"{initials}: {patients_did_move_n_per_dr[initials]}/{patients_n_per_dr[initials]}"
            if initials == "H":
                succeeded_count_str += f" + {extra_move_to_h} = {extra_move_to_h+patients_did_move_n_per_dr[initials]}/{patients_n_per_dr[initials]}\n"
            else:
                succeeded_count_str += "\n"
        print(succeeded_count_str)

    return moves


# move_df = pd.DataFrame(moves)
# move_df = move_df.loc[:, to_from_row_columns]
# print(f"Total patients: {len(move_df.index)}")
# move_df.sort_values(by="To", ascending=True, inplace=True)
    
# move_df.to_excel(filePath+"Moves.xlsx", index=False)


In [ ]:
wl_moves = pd.DataFrame(columns=to_from_row_columns)

for original_initials in ["JK", "MT", "MN", "DH"]:
    moves = divide(original_initials, time_periods)
    moves_df = pd.DataFrame(moves)
    wl_moves = wl_moves.append(moves_df, ignore_index=True)

In [ ]:
wl_moves.sort_values(by=["From", "To"], ascending=True, inplace=True)
wl_moves = wl_moves.loc[:, to_from_row_columns]
wl_moves.to_excel(filePath+"wl_moves.xlsx", index=False)

# Non waitinglist patients

In [ ]:
initials_with_wl = ["MT","JK","LW","RA","MS","MN","DH","AE","H"]
listingCounter = Counter(listingDf["Listing"])
total_wl_patients_except_hc = 0

for initials in initials_with_wl:
    print(f"{initials}\t{listingCounter[initials]}")

In [ ]:
wl_moves_personnr_set = set(wl_moves["Personnr"])
print(f"ListingDf before removing wl patients: {len(listingDf)}")
print(f"score Df before removing wl patients: {len(listedPatientScoreDf)}")
listingDfMinusWlPatients = listingDf.loc[ ~listingDf["Personnr"].isin(wl_moves_personnr_set) ]
scoringDfMinusWlPatients = listedPatientScoreDf.loc[ ~listedPatientScoreDf["Personnr"].isin(wl_moves_personnr_set) ]
print(f"ListingDf after removing wl patients: {len(listingDfMinusWlPatients)}")
print(f"score Df after removing wl patients: {len(scoringDfMinusWlPatients)}")
listing_minus_wl_counter = Counter(listingDfMinusWlPatients["Listing"])
print("\nLISTED PATIENTS WHEN WL PATIENTS ARE EXCLUDED")
for initials in initials_with_wl:
    print(f"{initials}\t{listing_minus_wl_counter[initials]}")

losing_on_wl_counter = Counter(wl_moves["From"])
gaining_on_wl_counter = Counter(wl_moves["To"])

print("\nEXPECTED LISTED PATIENTS AFTER WL TRANSFERS ARE COMPLETE")
for initials in initials_with_wl:
    print(f"{initials}\t{listingCounter[initials] + gaining_on_wl_counter[initials] - losing_on_wl_counter[initials]}")

In [ ]:
# NOTE: For listings not watilits
differences = {
    "MT": -100,
    "JK": -8,
    "LW": 29,
    "RA": -107,
    "MS": 291,
    "MN": -581,
    "DH": -622,
    "AE": 81,
    "H": 1005,
}

proportions_of_new = {
    "MT": 0,
    "JK": 0,
    "LW": 0.02060007424,
    "RA": 0,
    "MS": 0.2070414085,
    "MN": 0,
    "DH": 0,
    "AE": 0.05744898476,
    "H": 0.7149095325,
}

wl_moved_personnr = set(wl_moves["Personnr"])

In [ ]:
move_from_initials = ["MT", "JK", "RA", "MN", "DH"]
move_to_initials = ["LW", "MS", "AE", "H"]

In [ ]:
moves = []
fraction_to_save_for_original_when_moving_to_H = 0.4

for original_initials in move_from_initials:
    original_patients_left_set = set(listingDfMinusWlPatients.loc[ listingDfMinusWlPatients["Listing"] == original_initials ]["Personnr"])
    move_to_initials_left = move_to_initials.copy()
    move_to_initials_left.remove("H")
    move_total = abs(differences[original_initials])
    print(f"-----------------------\n#### {original_initials}\nMoving a total of {move_total}")

    print(f"{original_initials} has {len(original_patients_left_set)} initially, wl excluded")

    for initials in move_to_initials:
        move_to_n = round(move_total * proportions_of_new[initials])
        print(f"Moving {move_to_n} to {initials}")

        if initials != "H":
            if move_to_n == 0:
                print("None to move")
            else:
                personal_sorted_score_list = scoringDfMinusWlPatients.loc[ 
                    scoringDfMinusWlPatients["Personnr"].isin(original_patients_left_set) # Only patients listed on original
                ].sort_values(by=f"{initials}_score", ascending=False) # Sorted by target dr

                # If score is > for any of these, don't transfer the patietn.
                initials_to_compare = move_to_initials_left + [original_initials]
                moved_n = 0

                for index, row in personal_sorted_score_list.iterrows():
                    # Check if someone else should get it, including original dr.
                    exceeding = CheckIfOthersScoreExceeds(initials, row, 1.01)

                    # If a relevant dr exceeds:
                    if not exceeding or not listsHaveAnyMatches(exceeding, initials_to_compare):
                        personnr = row["Personnr"]
                        moved_n += 1
                        moves.append(create_to_from_row(row, original_initials, initials, category="listing"))
                        original_patients_left_set.remove(personnr)
                        if moved_n >= move_to_n:
                            break

                first_pass = moved_n

                # If note enoug have been moved, make a second round and move even those where the original dr score is higher
                if moved_n > move_to_n:
                    personal_sorted_score_list = personal_sorted_score_list.loc[ personal_sorted_score_list["Personnr"].isin(original_patients_left_set) ]
                    initials_to_compare = move_to_initials_left
                    for index, row in personal_sorted_score_list.iterrows():
                        # Check if someone else should get it, including original dr.
                        exceeding = CheckIfOthersScoreExceeds(initials, row, 1.01)

                        # If a relevant dr exceeds:
                        if not exceeding or not listsHaveAnyMatches(exceeding, initials_to_compare):
                            personnr = row["Personnr"]
                            moved_n += 1
                            moves.append(create_to_from_row(row, original_initials, initials, category="listing"))
                            original_patients_left_set.remove(personnr)
                            if moved_n >= move_to_n:
                                break

                print(f"Moved {moved_n} successfully ({first_pass} in the first pass)")

        else: # For H, just take a random sample except for the top X % of the originals score.
            personal_sorted_score_list = scoringDfMinusWlPatients.loc[ scoringDfMinusWlPatients["Personnr"].isin(original_patients_left_set) ]
            personal_sorted_score_list = personal_sorted_score_list.sort_values(by=f"{original_initials}_score", ascending=False)

            if original_initials in ["DH", "MN"]: # None should be left
                patients_to_move = personal_sorted_score_list
            else:
                n_rows = len(personal_sorted_score_list)
                start_index = int(n_rows * fraction_to_save_for_original_when_moving_to_H)  # Start index for bottom percent
                bottom_percent = personal_sorted_score_list.iloc[start_index:]
                patients_to_move = bottom_percent.sample(n=move_to_n)
            
            for index, row in patients_to_move.iterrows():
                moves.append(create_to_from_row(row, original_initials, initials, category="listing"))

            print(f"Moved {len(patients_to_move)} successfully to H using a random sample")

        if initials in move_to_initials_left:
            move_to_initials_left.remove(initials)

listing_moves_df = pd.DataFrame(moves)
listing_moves_df = listing_moves_df.loc[:, to_from_row_columns]

In [ ]:
listing_moves_df.to_excel(filePath+"listing_moves.xlsx", index=False)

In [ ]:
all_moves_df = wl_moves.append(listing_moves_df, ignore_index=True)

In [ ]:
all_moves_df = all_moves_df.sort_values(by=["From", "To", "Category", "total_score"], ascending=False)
all_moves_df.to_excel(filePath+"all_moves.xlsx", index=False)

# Check som stragglers

In [ ]:
# No moves from quiting dr to herself
all_moves_df.loc[ (all_moves_df["From"] == "MN") & (all_moves_df["To"] == "MN"), "To" ] = "H"
all_moves_df.loc[ (all_moves_df["From"] == "DH") & (all_moves_df["To"] == "DH"), "To" ] = "H"

In [ ]:
wl_moved_patients_set = set(wl_moves.loc[ wl_moves["From"] != wl_moves["To"] ]["Personnr"])
print(len(wl_moved_patients_set))
wl_no_duplicates = wait_list_df.drop_duplicates("Personnr")
non_moved_wl = wl_no_duplicates.loc[ ~wl_no_duplicates["Personnr"].isin(wl_moved_patients_set) ]
len(non_moved_wl)

In [ ]:
listing_dict = {}
for index, row in listingDf.iterrows():
    listing_dict[row["Personnr"]] = row["Listing"]

In [ ]:
all_moves_set = set(all_moves_df["Personnr"])
all_moves_adjusted_df = all_moves_df.reset_index(drop=True).copy() # For easier adding of new rows.

# Check for patients left on the waiting list, but not listed on the wl-dr
for initials in ["JK", "MT", "RA", "AE", "LW", "MS", "H"]:
    personal_wl = non_moved_wl.loc[ non_moved_wl["staff_member"] == initials ]
    print(f"-----------------------\n{initials} has {len(personal_wl)} left on the waitinglist")
    wrongly_listed_n = 0

    for index, row in personal_wl.iterrows():
        personnr = row["Personnr"]

        if personnr in listing_dict and listing_dict[personnr] != initials:
            # First make sure that the patient is not also on the other doctors waitinglist
            

            wrongly_listed_n += 1
            if personnr in all_moves_set:
                previous = all_moves_adjusted_df.loc[ all_moves_adjusted_df["Personnr"] == personnr, "To" ].iloc[0]
                print(f"Changed destination from {previous} to {initials}")
                all_moves_adjusted_df.loc[ all_moves_adjusted_df["Personnr"] == personnr, "To" ] = initials
            else:
                score_row = listedPatientScoreDf.loc[ listedPatientScoreDf["Personnr"] == personnr ].iloc[0]
                all_moves_adjusted_df.loc[len(all_moves_adjusted_df)] = create_to_from_row(score_row, listing_dict[personnr], initials)
                print(f"Added move from from {listing_dict[personnr]} to {initials}")


In [ ]:
all_moves_adjusted_df.to_excel(filePath+"all_moves_WITH_EXTRA_ADJUSTMENTS.xlsx", index=False)

In [ ]:
listingDf.columns

In [ ]:
# Move some extra from RA and LW to AE and H

moved_set = set(all_moves_adjusted_df["Personnr"])
def get_non_moved_for_dr(initials):
    personal_set = set(listingDf.loc[ listingDf["Listing"] == initials]["Personnr"])
    non_moved = listedPatientScoreDf.loc[ ~(listedPatientScoreDf["Personnr"].isin(moved_set)) & (listedPatientScoreDf["Personnr"].isin(personal_set)) ]
    non_moved_filtered = non_moved.loc[ non_moved[f"{initials}_score"] < 1.5 ]
    non_moved_filtered = non_moved_filtered.sort_values(by=f"total_score", ascending=False)
    move_rows = []
    for index, row in non_moved_filtered.iterrows():
        move_rows.append(create_to_from_row(row, initials, "", "listing"))
    df = pd.DataFrame(move_rows)
    df = df.loc[:, to_from_row_columns]
    return df

#get_non_moved_for_dr("LW")



In [ ]:
all_moves_adjusted_df = pd.read_excel(filePath+"all_moves_with_extra_adjustments_MANUALLY_EDITED.xlsx")

In [ ]:
moved_from_lw_set = set(all_moves_adjusted_df.loc[ (all_moves_adjusted_df["From"] == "LW") & (all_moves_adjusted_df["To"] != "LW")]["Personnr"])
moved_to_lw_set = set(all_moves_adjusted_df.loc[ (all_moves_adjusted_df["To"] == "LW")]["Personnr"])
len(moved_to_lw_set)

In [ ]:
# lw_patients_after_listing = set(listingDf.loc[ (listingDf["Listing"] == "LW") ]["Personnr"]) # Listed from start
# lw_patients_after_listing = (lw_patients_after_listing | moved_to_lw_set) - moved_from_lw_set
# len(lw_patients_after_listing)
# for personnr in lw_patients_after_listing:
#     print(personnr)

In [ ]:
len(lw_patients_after_listing)